In [1]:
import json
import tiktoken
import pandas as pd
import os
import re
from typing import Dict, Any

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def clean_string(s):
    """Clean and sanitize string for Excel."""
    if isinstance(s, str):
        s = ''.join(ch for ch in s if ord(ch) >= 32)
        s = s.replace('"', '""')
        s = re.sub(r'[\000-\010]|[\013-\014]|[\016-\031]', '', s)
        if s.startswith(('=', '+', '-', '@')):
            s = "'" + s
    return s

def flatten_dict(d: Dict[str, Any], parent_key: str = '', sep: str = '_') -> Dict[str, Any]:
    """Flatten nested dictionaries."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def results_json_to_excel(results_json, output_excel):
    """Convert posttest results JSON data to Excel format."""
    rows = []

    for teacher_name, teacher_data in results_json.items():
        teacher_config = teacher_data['config']
        for student_name, student_data in teacher_data.items():
            if student_name == 'config':
                continue
            student_config = student_data['config']
            for question_id, question_data in student_data.items():
                if question_id == 'config':
                    continue
                
                # Pre-test data
                pre_test_response = question_data['pre_test']['responses'][0]
                row = {
                    'teacher_student_pair': f"{teacher_name}_{student_name}",
                    'teacher_name': teacher_name,
                    'teacher_config_model': teacher_config.get('model', 'N/A'),
                    'teacher_config_education_theory': teacher_config.get('recommended_education_theory', 'N/A'),
                    'teacher_config_temperature': teacher_config.get('temperature', 'N/A'),
                    'teacher_config_max_tokens': teacher_config.get('max_tokens', 'N/A'),
                    'teacher_use_few_shot': teacher_config.get('use_few_shot', 'N/A'),
                    'teacher_recommended_question_token_limit': teacher_config.get('recommended_question_token_limit', 'N/A'),
                    'teacher_max_tokens_rerun_threshold_percentage': teacher_config.get('max_tokens_rerun_threshold_percentage', 'N/A'),
                    'student_name': student_name,
                    'student_config_model': student_config.get('model', 'N/A'),
                    'student_config_temperature': student_config.get('temperature', 'N/A'),
                    'student_config_answer_max_tokens': student_config.get('answer_max_tokens', 'N/A'),
                    'student_config_test_max_tokens': student_config.get('test_max_tokens', 'N/A'),
                    'student_use_few_shot': student_config.get('use_few_shot', 'N/A'),
                    'student_include_pretest_info': student_config.get('include_pretest_info', 'N/A'),
                    'student_recommended_answer_token_limit': student_config.get('recommended_answer_token_limit', 'N/A'),
                    'student_recommended_test_token_limit': student_config.get('recommended_test_token_limit', 'N/A'),
                    'student_max_tokens_rerun_threshold_percentage': student_config.get('max_tokens_rerun_threshold_percentage', 'N/A'),
                    'question_id': question_id,
                    'category': question_data.get('category', 'N/A'),
                    'question': pre_test_response.get('question', 'N/A'),
                    'options': json.dumps(pre_test_response.get('options', [])),
                    'correct_answer': pre_test_response.get('correct_answer', 'N/A'),
                    'pre_test_model_response': pre_test_response.get('model_response', 'N/A'),
                    'pre_test_model_prediction': pre_test_response.get('model_prediction', 'N/A'),
                    'pre_test_model_accuracy': int(pre_test_response.get('correct_answer', '') == pre_test_response.get('model_prediction', '')),
                    'pre_test_model_response_tokens': num_tokens_from_string(question_data['pre_test']['responses'][0].get('model_response', ''), "cl100k_base"),
                }

                # Process interactions
                for i, interaction in enumerate(question_data['interaction'], 1):
                    row[f'interaction_{i}_question'] = interaction['question']
                    row[f'interaction_{i}_answer'] = interaction['answer']
                    row[f'interaction_{i}_question_tokens'] = num_tokens_from_string(interaction['question'], "cl100k_base")
                    row[f'interaction_{i}_answer_tokens'] = num_tokens_from_string(interaction['answer'], "cl100k_base")

                # Process post_test
                row.update({
                    'post_test_model_response': question_data['post_test']['responses'][0].get('model_response', 'N/A'),
                    'post_test_model_prediction': question_data['post_test']['responses'][0].get('model_prediction', 'N/A'),
                    'post_test_model_accuracy': int(question_data['post_test']['responses'][0].get('correct_answer', '') == question_data['post_test']['responses'][0].get('model_prediction', '')),
                    'post_test_model_response_tokens': num_tokens_from_string(question_data['post_test']['responses'][0].get('model_response', ''), "cl100k_base"),
                })

                rows.append(flatten_dict(row))

    df = pd.DataFrame(rows)

    # Add this check to verify the data
    print(df[['question_id', 'category', 'question', 'options', 'correct_answer', 'pre_test_model_prediction', 'post_test_model_prediction']].head())

    for column in df.columns:
        df[column] = df[column].apply(clean_string)

    df.to_excel(output_excel, index=False, engine='openpyxl')
    print(f"Excel file saved as: {output_excel}")

def main():
    file_path = 'D:/Workspace/EducationQ_Benchmark/src/data/output/EduQ-Bench_Student-mistral-nemo/GPQA/'
    file_name = "pretest_interaction_posttest_results_1.0.0_20241018_190120"
    #json_file_path = os.path.join(file_path, file_name + ".json")
    #excel_file_path = os.path.join(file_path, file_name + '_with_tokens.xlsx')

    #print("Processing JSON and counting tokens...")
    #processed_data = process_json(json_file_path)


    print("Converting to Excel...")
    #results_json_to_excel(json_file_path, excel_file_path)

# 将posttest_results的json转成excel表
    posttest_results_file = os.path.join(file_path, file_name + ".json")
    output_excel_file = os.path.join(file_path, file_name + '.xlsx')
    with open(posttest_results_file, "r") as f:
        posttest_results = json.load(f)
    
    results_json_to_excel(posttest_results, output_excel_file)

    print("Process completed successfully!")

if __name__ == "__main__":
    main()

Converting to Excel...
         question_id category  \
0  rec0yTRmO1o1xCA6H  Physics   
1  rec260hNUCEj109Dj  Physics   
2  rec0wZvZgiz320KRs  Physics   
3  rec1zl5LvaatzGhFt  Physics   
4  rec5eA17Qr1ucNLUf  Physics   

                                            question  \
0  In a parallel universe where a magnet can have...   
1  A quantum mechanical particle of mass m moves ...   
2  A spin-half particle is in a linear superposit...   
3  Calculate the eigenvector of a quantum mechani...   
4  Researchers are attempting to detect transits ...   

                                             options correct_answer  \
0  ["The ones related to the divergence and the c...              C   
1  ["E = (n_x+3*n_y+3/2) \u210f*sqrt(k/m))", "E =...              D   
2                   ["0.85", "1.65", "-1.4", "-0.7"]              D   
3  ["(\\sqrt{2/3}\\hbar, \\sqrt{1/3}\\hbar)", "(\...              D   
4  ["Planet_1 is preferred due to its ~1.65 times...              A   

  pre_test_mod